It is tricky to write high precision C code to calculate steady-states for the polymerase-cycle model with multiple TF sites for 1 TF. If one tries to do it with the write_multiplebsites.write_pybind_module_highprecision, it fails to compile with more than 3 sites. It seems that the issue is solved if one uses the sparse matrix approach of write_multiplebsites.write_pybind_module. In the end it can just be done with Python. Keeping this for the record.

In [1]:
import numpy as np
import sys, os,re
#path_to_eigen="/Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/"
path_to_eigen="/Users/rosamartinezcorral/Documents/eigen-3.4.0/Eigen"
path_to_eigen="/Users/rosamartinezcorral/Documents/eigen-3.4.0" 
path_to_utilssynpy="./repos_commit/scriptssyn/"
path_to_utilsGRF="./repos_commit/GeneRegulatoryFunctions/utilsGRF/"

sys.path.append(path_to_utilssynpy)
import write_multiplebsites #in this case the usage of spanning trees becomes impractical. Therefore, in this code, standard routines to compute the matrix nullspace are used


In [5]:
#Now make code with 4 binding sites.
#Note that this works with the write_pybind_module, which uses double types. If one tries write_pybind_module_highprecision, it fails to compile
#In order to solve this:
#1 execute the write_pybind_module
#2 execute the write_pybind_module_higherprec
#3 modify the file generated in 2 with

TFnames=["x"]
#independent binding of TFs to each site and pol cycle state
for bsites in [4,6]:
    
    
    print(bsites,TFnames)
    nnodes_cycle=3
    rev_cycle=[1]

    pars,nnodes,L_string,L_string_new,node_indices,transitions,alledges=write_multiplebsites.get_edges_anyindepsites_1TF(sites=bsites,nnodes_cycle=nnodes_cycle,rev_cycle=rev_cycle)
    fname="./bin/base_%dsites_highprec.cpp"%bsites
    
    file=fname
    objectnamemac=fname.replace(".cpp","")
    prec=50
    write_multiplebsites.write_pybind_module_highprecision(fname,pars,nnodes,TFnames,L_string_new,bsites,indicesC=node_indices,coeffsC=transitions,precision=prec)

    fname="./bin/base_%dsites_double.cpp"%bsites
    file=fname
    objectnamemac=fname.replace(".cpp","")
    write_multiplebsites.write_pybind_module(fname,pars,nnodes,TFnames,L_string,bsites,indicesC=node_indices,coeffsC=transitions)

    

4 ['x']
binding graph:
[[1, 'a1_0-x', 2], [2, 'b1_1', 1], [1, 'a2_0-x', 3], [3, 'b2_2', 1], [1, 'a3_0-x', 4], [4, 'b3_3', 1], [1, 'a4_0-x', 5], [5, 'b4_4', 1], [2, 'a2_1-x', 6], [6, 'b2_1U2', 2], [2, 'a3_1-x', 7], [7, 'b3_1U3', 2], [2, 'a4_1-x', 8], [8, 'b4_1U4', 2], [3, 'a1_2-x', 6], [6, 'b1_1U2', 3], [3, 'a3_2-x', 9], [9, 'b3_2U3', 3], [3, 'a4_2-x', 10], [10, 'b4_2U4', 3], [4, 'a1_3-x', 7], [7, 'b1_1U3', 4], [4, 'a2_3-x', 9], [9, 'b2_2U3', 4], [4, 'a4_3-x', 11], [11, 'b4_3U4', 4], [5, 'a1_4-x', 8], [8, 'b1_1U4', 5], [5, 'a2_4-x', 10], [10, 'b2_2U4', 5], [5, 'a3_4-x', 11], [11, 'b3_3U4', 5], [6, 'a3_1U2-x', 12], [12, 'b3_1U2U3', 6], [6, 'a4_1U2-x', 13], [13, 'b4_1U2U4', 6], [7, 'a2_1U3-x', 12], [12, 'b2_1U2U3', 7], [7, 'a4_1U3-x', 14], [14, 'b4_1U3U4', 7], [8, 'a2_1U4-x', 13], [13, 'b2_1U2U4', 8], [8, 'a3_1U4-x', 14], [14, 'b3_1U3U4', 8], [9, 'a1_2U3-x', 12], [12, 'b1_1U2U3', 9], [9, 'a4_2U3-x', 15], [15, 'b4_2U3U4', 9], [10, 'a1_2U4-x', 13], [13, 'b1_1U2U4', 10], [10, 'a3_2U4-x', 15]

Now manually modify the files following the instructions in "whattochange_highprec_multiplesites" and compile:


In [15]:
#independent binding of TFs to each site and pol cycle state
for bsites in [4,6]:
    
    fname="./bin/final_%dsites_highprec.cpp"%bsites
    
    file=fname
    objectnamemac=fname.replace(".cpp","")
    compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I %s -I /opt/homebrew/include -I /Users/rosamartinezcorral/Documents/eigen-3.4.0 -L /opt/homebrew/lib -lmpfr -lmpc `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(path_to_utilsGRF,file,objectnamemac)
    print(compilestringmac)
    !$compilestringmac

c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I ./repos_commit/GeneRegulatoryFunctions/utilsGRF/ -I /opt/homebrew/include -I /Users/rosamartinezcorral/Documents/eigen-3.4.0 -L /opt/homebrew/lib -lmpfr -lmpc `python3 -m pybind11 --includes` ./bin/final_6sites_highprec.cpp -o ./bin/final_6sites_highprec`python3-config --extension-suffix`
In file included from ./bin/final_6sites_highprec.cpp:3:
In file included from /Users/rosamartinezcorral/miniconda3/envs/torchEnv/lib/python3.10/site-packages/pybind11/include/pybind11/eigen.h:12:
In file included from /Users/rosamartinezcorral/miniconda3/envs/torchEnv/lib/python3.10/site-packages/pybind11/include/pybind11/eigen/matrix.h:27:
In file included from /Users/rosamartinezcorral/Documents/eigen-3.4.0/Eigen/SparseCore:61:
/Users/rosamartinezcorral/Documents/eigen-3.4.0/Eigen/src/SparseCore/TriangularSolver.h:273:13: warning: variable 'count' set but not used [-Wunused-but-set-variable]
      Index count = 0;
       

./bin/final_6sites_highprec.cpp:49:9: warning: unused variable 'n' [-Wunused-variable]
    int n=192;
        ^
9 warnings generated.
